In [22]:
import pandas as pd 
from IPython.display import display
test = True

We are going to explore data from the United Nations with the following post as a guideline. 

_Hannah Ritchie and Max Roser (2018) - "Urbanization"._
</br>_Published online at OurWorldInData.org._
</br>[https://ourworldindata.org/urbanization](https://ourworldindata.org/urbanization)

## Current situation

The current urban and rural populations per country can be retrieved from the United Nations data portal as indicated in the article.

In [2]:
url = 'https://population.un.org/wup/Download/Files/WUP2018-F01-Total_Urban_Rural.xls'

In [3]:
df_tur_raw = pd.read_excel(io=url)

In [4]:
# The first 15 lines are the headers of the Excel sheet
df_tur = df_tur_raw[15:].copy()
df_tur.columns = df_tur_raw.loc[14].str.replace(',', '').str.replace(' ', '_').str.replace('\n', '_').str.lower().tolist()
df_tur.drop(labels=['index', 'note', 'total', 'percentage_urban'], axis='columns', inplace=True)

In [5]:
df_tur['country_code'] = df_tur.country_code.astype(int)
# Population are by thousands
for c in ['urban', 'rural']:
    df_tur[c] = df_tur[c] * 1000
    df_tur[c] = df_tur[c].astype(int)

In [23]:
if test:
    df_tur.head(n=10).pipe(display)

,region_subregion_country_or_area,country_code,urban,rural,region,sub_region
15,WORLD,900,4219817318,3413002007,,
16,More developed regions,901,993837090,269362587,,
17,Less developed regions,902,3225980228,3143639420,,
18,Least developed countries,941,344291432,681645301,,
19,"Less developed regions, excluding least develo...",934,2881688796,2461994118,,
20,"Less developed regions, excluding China",948,2362378537,2560439789,,
21,High-income countries,1503,975461275,221729757,,
22,Middle-income countries,1517,3017642113,2719999603,,
23,Upper-middle-income countries,1502,1759117809,881117009,,
24,Lower-middle-income countries,1501,1258524303,1838882593,,


In [7]:
def assign_groups(labels, groups):
    current_label = ''
    local_groups = []
    for label in labels:
        current_label = label if label in groups else current_label
        local_groups.append(current_label)
        
    return local_groups

In [8]:
regions = ['AFRICA', 'LATIN AMERICA AND THE CARIBBEAN', 'NORTHERN AMERICA', 'EUROPE', 'OCEANIA', 'ASIA']
df_tur['region'] = assign_groups(
    labels=df_tur.region_subregion_country_or_area.tolist(),
    groups=regions)

In [24]:
if test:
    df_tur[df_tur.country_code > 902][['region_subregion_country_or_area', 'country_code']].pipe(display)

,region_subregion_country_or_area,country_code
18,Least developed countries,941
19,"Less developed regions, excluding least develo...",934
20,"Less developed regions, excluding China",948
21,High-income countries,1503
22,Middle-income countries,1517
23,Upper-middle-income countries,1502
24,Lower-middle-income countries,1501
25,Low-income countries,1500
26,Sub-Saharan Africa,947
27,AFRICA,903


In [25]:
# Extract
more_than_903 = df_tur.country_code > 902
in_regions = df_tur.region_subregion_country_or_area.isin(regions)
not_sub_regions = df_tur.region_subregion_country_or_area.str.match('.*(countries|Less|More)')

sub_regions = df_tur[more_than_903 & ~in_regions & ~not_sub_regions].region_subregion_country_or_area.tolist()
df_sub_regions = df_tur[
    df_tur.region_subregion_country_or_area.isin(sub_regions)
].groupby(by='region').region_subregion_country_or_area.apply(lambda s: ', '.join(s.tolist()))

if test:
    df_sub_regions.pipe(display)

region
                                                                  Sub-Saharan Africa
AFRICA                             Eastern Africa, Middle Africa, Northern Africa...
ASIA                               Eastern Asia, South-Central Asia, Central Asia...
EUROPE                             Eastern Europe, Northern Europe, Southern Euro...
LATIN AMERICA AND THE CARIBBEAN            Caribbean, Central America, South America
OCEANIA                            Australia/New Zealand, Melanesia, Micronesia, ...
Name: region_subregion_country_or_area, dtype: object

In [13]:
df_tur['sub_region'] = assign_groups(
    labels=df_tur.region_subregion_country_or_area.tolist(),
    groups=sub_regions)
df_tur.loc[df_tur.region == 'NORTHERN AMERICA', 'sub_region'] = 'Northern America'

In [14]:
# Countries have a country code lower than 900
df_c = df_tur[df_tur.country_code < 900].copy()
df_c.rename(columns=dict(region_subregion_country_or_area='country'), inplace=True)

In [27]:
if test:
    df_c[~df_c.region.isin(regions)].pipe(display)
    df_c[df_c.sub_region.isnull()].pipe(display)
    df_c[df_c.country.isin(['France', 'Malaysia'])].pipe(display)

,country,country_code,urban,rural,region,sub_region


,country,country_code,urban,rural,region,sub_region


,country,country_code,urban,rural,region,sub_region
123,Malaysia,458,24363730,7678728,ASIA,South-Eastern Asia
195,France,250,52476365,12756906,EUROPE,Western Europe


In [28]:
# We can now compute the total by region
if test:
    df_r = df_c.groupby(by='region').agg(
        dict(
            urban='sum',
            rural='sum'))

    df_rg = df_tur[
        df_tur.region_subregion_country_or_area.isin(regions)
    ][['urban', 'rural', 'region_subregion_country_or_area']].set_index('region_subregion_country_or_area')

    df_r.join(df_rg, rsuffix='_g').pipe(display)

,urban,rural,urban_g,rural_g
region,,,,
AFRICA,547602180,740318336,547602182,740318335
ASIA,2266130580,2279002514,2266130580,2279002514
EUROPE,552911224,189736785,552911225,189736785
LATIN AMERICA AND THE CARIBBEAN,526057192,125954809,526057192,125954808
NORTHERN AMERICA,298987110,64857380,298987110,64857380
OCEANIA,28129027,13132183,28129029,13132183
